# 2016 Georgia Election Comparison - Data Cleaning and Exploratory Analysis

Alexander Church

DS 4003

### Goal of project

The goal of the dashboard is to compare the results in congressional districts in Georgia in House elections to presidential elections.

### Data provenence

The data comes from the Election Data and Science Lab at the Massachusetts Institute of Technology. The data comes from two files, [2016-precinct-president.csv](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/LYWX3D) and [2016-precinct-house.tab](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PSKDUJ).

### Description of data

The data describes election results in the 2016 federal election, for the US House race and the US Presidential race (between Donald Trump and Hillary Clintion). The data was filtered to just cover the state of Georgia, as the entire country would be unfeasible for the project. The data has information by precinct, as well as details such as the method of voting (same-day, early, mail, etc.) which can lead to interesting analysis.

## Data cleaning:

In [28]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from dash import Dash, html, dcc 
import geopandas as gpd
import json
import pandas_geojson as pdg

In [29]:
# Import presidential data
pres2016 = pd.read_csv('2016-precinct-president.csv', encoding='ISO 8859-1')

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\46099483.py:2: DtypeWarning:

Columns (13,17,19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.



In [30]:
# Drop needless columns
columns_to_drop = ['stage', 'state_fips','state_icpsr','county_fips',  'county_lat','county_long','office','candidate_last', 'candidate_first', 'candidate_middle', 'candidate_full', 'candidate_suffix', 'candidate_nickname','candidate_google', 'candidate_govtrack', 'candidate_icpsr', 'candidate_maplight', 'candidate_opensecrets', 'candidate_wikidata', 'candidate_fec', 'candidate_party']
pres2016 = pres2016.drop(columns = columns_to_drop)

pres2016['precinct'] = pres2016['precinct'].str.lower()

# Filter out write-in candidates
pres2016 = pres2016[pres2016['writein'] == False]

# Create precinct ID
pres2016['precinctid'] = pres2016['precinct'].astype('string') + pres2016['jurisdiction'].astype('string')

# Filter to Georgia
pres2016 = pres2016[pres2016['state'] == 'Georgia']

# New column for 'race' - which election
pres2016['race'] = 'pres'

# Replace parties
presparties = {'Hillary Clinton': 'democratic', 'Gary Johnson': 'libertarian', 'Donald Trump': 'republican'}
pres2016['party'] = pres2016['candidate'].map(presparties)

# Replace district
districtmap = {'pres': 0}
pres2016['district'] = pres2016['race'].map(districtmap)

# Reset index
pres2016 = pres2016.reset_index(drop=True)

# Clean certain value
pres2016 = pres2016.replace('absentee by mail', 'mail')
pres2016 = pres2016.replace('advance in person', 'early')

In [31]:
# Import house data
house2016 = pd.read_csv('2016-precinct-house.tab', encoding='ISO 8859-1')

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\621711366.py:2: DtypeWarning:

Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.



In [32]:
# Filter to Georgia
house2016 = house2016[house2016['state'] == 'Georgia']

# Drop needless columns
house2016 = house2016.drop(columns = columns_to_drop)

house2016['precinct'] = house2016['precinct'].str.lower()

# Filter out write-in candidates
house2016 = house2016[house2016['writein'] == False]

# Create precinct ID
house2016['precinctid'] = house2016['precinct'] + house2016['jurisdiction']

# New column for 'race' - which election
house2016['race'] = 'house'

# Exclude certain type of data
house2016 = house2016[house2016['mode'] != 'total']

house2016['district'] == house2016['district'].astype('int').astype('str')

# Reset index
house2016 = house2016.reset_index(drop=True)

# Clean certain candidates
house2016 = house2016.replace('EARL â\x80\x9cBUDDYâ\x80\x9d CARTER', 'Buddy Carter')
house2016 = house2016.replace('HENRY C â\x80\x9cHANKâ\x80\x9d JOHNSON JR', 'Henry Johnson Jr.')
house2016.loc[house2016['candidate'].str.contains('Carter'), 'party'] = 'republican'
house2016 = house2016.replace('absentee by mail', 'mail')
house2016 = house2016.replace('advance in person', 'early')

In [33]:
#Combine dataframes, reset index, drop needless columns
all2016 = pd.concat([pres2016, house2016]).reset_index(drop = True).drop(
    columns = ['special', 'state_postal', 'county_ansi', 'candidate_normalized', 'candidate_fec_name', 'writein'])

all2016.columns

Index(['year', 'state', 'county_name', 'jurisdiction', 'precinct', 'candidate',
       'district', 'party', 'mode', 'votes', 'precinctid', 'race'],
      dtype='object')

In [34]:
all2016

,year,state,county_name,jurisdiction,precinct,candidate,district,party,mode,votes,precinctid,race
0,2016,Georgia,Appling County,Appling,2,Hillary Clinton,0.0,democratic,election day,338,2Appling,pres
1,2016,Georgia,Appling County,Appling,2,Hillary Clinton,0.0,democratic,mail,87,2Appling,pres
2,2016,Georgia,Appling County,Appling,2,Hillary Clinton,0.0,democratic,early,317,2Appling,pres
3,2016,Georgia,Appling County,Appling,2,Hillary Clinton,0.0,democratic,provisional,0,2Appling,pres
4,2016,Georgia,Appling County,Appling,2,Gary Johnson,0.0,libertarian,election day,5,2Appling,pres
...,...,...,...,...,...,...,...,...,...,...,...,...
50287,2016,Georgia,Worth County,Worth,warwick,JAMES NEAL HARRIS,8.0,democratic,provisional,0,warwickWorth,house
50288,2016,Georgia,Worth County,Worth,warwick,AUSTIN SCOTT,8.0,republican,election day,212,warwickWorth,house
50289,2016,Georgia,Worth County,Worth,warwick,AUSTIN SCOTT,8.0,republican,mail,13,warwickWorth,house
50290,2016,Georgia,Worth County,Worth,warwick,AUSTIN SCOTT,8.0,republican,early,72,warwickWorth,house


In [35]:
house2020 = pd.read_csv('HOUSE_precinct_general.csv', encoding='ISO 8859-1')

In [36]:
# Filter to Georgia
house2020 = house2020[house2020['state'] == 'GEORGIA']

house2020 = house2020[house2020['writein'] == False]

dropcolumns = ['party_detailed', 'county_fips', 'jurisdiction_name', 'jurisdiction_fips', 'dataverse', 'stage', 'special', 'writein', 'state_po', 'state_fips', 'state_cen', 'state_ic', 'date', 'readme_check', 'magnitude']
house2020 = house2020.drop(columns = dropcolumns)

house2020['race'] = 'house'

# Change capitalization to match 2016 data sets
house2020['party'] = house2020['party_simplified'].str.lower()
house2020['mode'] = house2020['mode'].str.lower()
house2020['jurisdiction'] = house2020['county_name'].str.lower().str.capitalize()

# Clean certain values
house2020 = house2020.replace('Mcintosh', 'McIntosh')
house2020 = house2020.replace('Dekalb', 'DeKalb')
house2020 = house2020.replace('Ben hill', 'Ben Hill')
house2020 = house2020.replace('Mcduffie', 'McDuffie')
house2020 = house2020.replace('Jeff davis', 'Jeff Davis')

# Change capitalization to match 2016 data sets
house2020['precinct'] = house2020['precinct'].str.lower().str.capitalize()
house2020['county_name'] = house2020['jurisdiction'].str.capitalize() + ' County'
house2020['state'] = 'Georgia'

# Create precinct id
house2020['precinctid'] = house2020['precinct'] + house2020['jurisdiction']

# Drop some columns
house2020 = house2020.drop(columns = ['party_simplified', 'office'])

# Clean certain values
house2020 = house2020.replace('absentee', 'mail')
house2020 = house2020.replace('advanced voting', 'early')
house2020 = house2020.replace('democrat', 'democratic')

In [37]:
pres2020 = pd.read_csv('PRESIDENT_precinct_general.csv', encoding = 'ISO 8859-1')

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\2680817445.py:1: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



In [38]:
# Filter to Georgia
pres2020 = pres2020[pres2020['state'] == 'GEORGIA']

pres2020 = pres2020[pres2020['writein'] == False]

pres2020 = pres2020.drop(columns = dropcolumns)

pres2020['race'] = 'pres'
pres2020['district'] = 0

# Change capitalization to match 2016 data sets
pres2020['party'] = pres2020['party_simplified'].str.lower()
pres2020['mode'] = pres2020['mode'].str.lower()
pres2020['jurisdiction'] = pres2020['county_name'].str.lower().str.capitalize()

# Clean certain values
pres2020 = pres2020.replace('Mcintosh', 'McIntosh')
pres2020 = pres2020.replace('Dekalb', 'DeKalb')
pres2020 = pres2020.replace('Ben hill', 'Ben Hill')
pres2020 = pres2020.replace('Mcduffie', 'McDuffie')
pres2020 = pres2020.replace('Jeff davis', 'Jeff Davis')

# Change capitalization to match 2016 data sets
pres2020['precinct'] = pres2020['precinct'].str.lower().str.capitalize()
pres2020['county_name'] = pres2020['jurisdiction'].str.capitalize() + ' County'
pres2020['state'] = 'Georgia'

# Create precinct id
pres2020['precinctid'] = pres2020['precinct'] + pres2020['jurisdiction']

# Drop some columns
pres2020 = pres2020.drop(columns = ['party_simplified', 'office'])

# Clean certain values
pres2020 = pres2020.replace('JO JORGENSEN', 'Jo Jorgensen')
pres2020 = pres2020.replace('DONALD J TRUMP', 'Donald Trump')
pres2020 = pres2020.replace('JOSEPH R BIDEN', 'Joseph Biden')
pres2020 = pres2020.replace('democrat', 'democratic')
pres2020 = pres2020.replace('absentee', 'mail')
pres2020 = pres2020.replace('advanced voting', 'early')

In [39]:
pres2020['party'].unique()

array(['democratic', 'libertarian', 'republican'], dtype=object)

In [40]:
alldata = pd.concat([all2016, house2020, pres2020]).reset_index(drop = True)

# Clean data type

alldata['district'] = pd.to_numeric(alldata['district'])

In [59]:
dfff = pd.DataFrame(columns = ['district', 'year', 'reppercent'])

for j in [2016, 2020]:
    for i in range(0, 15):
        x = {'district': i, 'year': j, 'reppercent': (sum(alldata[alldata['district'] == i][alldata['year'] == j][alldata['party'] == 'republican']['votes'])) / (sum(alldata[alldata['district'] == i][alldata['year'] == j][alldata['party'] == 'republican']['votes']) + sum(alldata[alldata['district'] == i][alldata['year'] == j][alldata['party'] == 'democratic']['votes']))}
        dfff = pd.concat([dfff, pd.DataFrame([x])], ignore_index = True)

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\3100047441.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\3100047441.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\3100047441.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\3100047441.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\3100047441.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\3100047441.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\3100047441.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame 

## Data dictionary:

| Variable | Full Name | Description |
|----------|-----------|-------------|
|year|Year|Year of Election (always 2016)|
|state|State|State of Election (always Georgia)|
|county_name|County Name|Name of County of Election|
|jurisdiction|Jurisdiction|Jurisdiction running election (always name of county)|
|precinct|Precinct|Precinct of election|
|candidate|Candidate|Name of candidate|
|district|District|House district of precinct ('statewide' for presidential rows)|
|party|Political Party|Political party of candidate|
|mode|Voting Mode|Mode of Voting (election day, mail, etc.)|
|votes|Votes|Number of votes in precinct|
|precinctid|Precinct ID|Unique ID for Precinct, precinct name concatenated with jurisdiction|
|race|Race|Election candidate is running in (house or pres)|

## Exploratory data analysis:

### Summary of data

In [41]:
## Precinct counts

precinctshouse2016 = alldata[alldata['race'] == 'house'][alldata['year'] == 2016]['precinctid'].unique() # House precincts
precinctspres2016 = alldata[alldata['race'] == 'pres'][alldata['year'] == 2016]['precinctid'].unique() # Pres precincts
precinctshouse2020 = alldata[alldata['race'] == 'house'][alldata['year'] == 2020]['precinctid'].unique() # House precincts
precinctspres2020 = alldata[alldata['race'] == 'pres'][alldata['year'] == 2020]['precinctid'].unique() # Pres precincts

print(len(precinctshouse2016))
print(len(precinctspres2016))
print(len(precinctshouse2020))
print(len(precinctspres2020))

# Summary of dataset
print(alldata.head())

2693
2693
2656
2656
   year    state     county_name jurisdiction precinct        candidate  \
0  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
1  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
2  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
3  2016  Georgia  Appling County      Appling        2  Hillary Clinton   
4  2016  Georgia  Appling County      Appling        2     Gary Johnson   

   district        party          mode  votes precinctid  race  
0       0.0   democratic  election day    338   2Appling  pres  
1       0.0   democratic          mail     87   2Appling  pres  
2       0.0   democratic         early    317   2Appling  pres  
3       0.0   democratic   provisional      0   2Appling  pres  
4       0.0  libertarian  election day      5   2Appling  pres  


C:\Users\achur\AppData\Local\Temp\ipykernel_5780\1262917838.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\1262917838.py:4: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\1262917838.py:5: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\1262917838.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



The data shows 2,693 precincts in Georgia for the House election and the presidential election in 2016, and 2,656 for the elections in 2020. Georgia has 159 counties and 14 districts, so this makes approximately 16.7-16.9 precincts per county and 190-192 precincts per congressional district.

The only continuous variable in the dataset is votes. Therefore, all representations of the data will be filtered and divided, but only have one output variable (votes). This limits the amount of representations, but forces the dashboard to be creative.

### Vote counts per race

In [42]:
# Filter presidential data
presrace = alldata[alldata['district'] == 'statewide']

# Filter house data
houseraces = alldata[alldata['race'] == 'house']

# Create grouped bar graph
fig1 = px.histogram(houseraces, 
                   x = "district",  # by district
                   y = "votes", # total votes per party
                   color = 'party', # republican or democratic
                   barmode = 'group',
                   facet_row="year",
                   histfunc = 'sum', # sum of votecount
                   title = 'Results in Georgia Congressional Districts',
                   color_discrete_map = {"republican": "red", "democratic": "blue"}, # matching colors
                   height=800)

fig1.show() # Display figure

In [43]:
fig2 = px.histogram(
            alldata[alldata['mode'] != 'provisional'][alldata['party'] != 'libertarian'], # Dataframe filtering
             x="party", # by party
             y="votes",
             color="party",
             facet_col="mode", # seperate by mode of voting
             facet_row = 'year',
             histfunc = 'sum', # sum of votecount
             title = 'Results by Voting Type',
             color_discrete_map = {"republican": "red", "democratic": "blue"}, # matching colors
             category_orders={"race": ["pres", "house"]})

fig2.show()

C:\Users\achur\AppData\Local\Temp\ipykernel_5780\2135392253.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Interestingly, the party split roughly stays the same between different modes of voting. Specifically after the politicization of mail-in voting in the 2020 election, it is expected that more Democrats engage in mail-in voting than Republicans; however, that assumption does not hold for this election. There was also much less mail-in voting in 2016 than 2020, mostly due to the COVID-19 pandemic.

### Possible List of UI Components

- Dropdown to filter between House districts
- Radio buttons to filter House vs President
- Checkboxes to filter voting mode
- Dropdown to filter between candidates
- With more data (which I may organize and include), slider from 2016 to 2020

### Possible Data Visualizations

- Geographic representation of results, with districts shaded different colors
    - Can be filtered for two different elections, or side-by-side
- Individual district results broken down by:
    - Voting mode
    - Comparison between house results and Presidential results
    - With more data (which I may organizae and include), trends from 2016 to 2020

### Export to CSV (commented out):

In [60]:
alldata.to_csv('data.csv', index=False)

dfff.to_csv('reppercent.csv', index = False)